Input: arquivo com a mensagem em claro + Chave RSA pública do destinatário + Algoritmo simétrico

In [ ]:
from Crypto.PublicKey import RSA

In [ ]:
def generateKeyRSA(secretCode:str, legthModuleRSA:int) -> bytes:
    key = RSA.generate(legthModuleRSA)
    encriptedKey = key.export_key(passphrase=secretCode, pkcs=8, protection="scryptAndAES128-CBC")
    return encriptedKey

In [ ]:
def generatePairKeys(length): return RSA.newKeys(length) #Gera par de chaves publica e privada
def generateRandomKey(length): return RSA.generate(length) 

In [ ]:
def writeKey(key:bytes, mode:int) -> bool:
    if mode == 0:
        with open("./input/rsaKey.bin", "wb") as archive:
            try:
                archive.write(key)
                return True
            except:
                return False
    elif mode == 1:
        with open("./input/private.pem", "wb") as archive:
            try:
                archive.write(key)
                return True
            except:
                return False
    elif mode == 2:
        with open("./input/receiver.pem", "wb") as archive:
            try:
                archive.write(key)
                return True
            except:
                return False

In [ ]:
def main():
    rsa = generateKeyRSA("cachaça", 2048)
    private = generateKeyPrivate(2048)
    public = generateKeyPublic(2048)
    writeKey(rsa, 0)
    writeKey(private, 1)
    writeKey(public, 2)


if __name__ == "__main__":
    main()